**Общая информация**

**Срок сдачи:** 13 марта 2017, 06:00 <br\>
**Штраф за опоздание:** -2 балла после 06:00 13 марта, -4 балла после 06:00 20 марта, -6 баллов после 06:00 27 марта

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack создать таск в приватный чат:
/todo Фамилия Имя ссылка на гитхаб @alkhamush
Пример:
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе.

###### Задание 2 (2 балла)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn 4 балла для iris и mnist
Для этого используем numpy

###### Задание 3 (2 балла)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 3 (2 балла)
Выполнить требования pep8

###### Задание 5 (2 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
%load_ext pycodestyle_magic

In [9]:
#%%pycodestyle

class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin, ClassifierMixin):

    C = 100

    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        self._X = X.copy().astype(float)
        self._X_ms = ((self._X ** 2).sum(axis = 1)) ** 0.5
        self._y = y.copy().astype(int)
        self.classes = np.unique(y)

    def predict(self, X):
        return self.predict_proba(X).argmax(axis = 1)

    def predict_proba(self, X):
        cnt = np.zeros([X.shape[0], self.classes.shape[0]])
        for i in range(np.ceil(X.shape[0] / float(self.C)).astype(int)):
            X_c = X[self.C * i: self.C * (i + 1)].astype(float)
            X_m2 = (X_c ** 2).sum(axis = 1)
            dst = X_c.dot(self._X.T)
            dst = -2 * dst + self._X_ms ** 2 + X_m2.reshape(-1, 1)
            dst_m = self._y[dst.argsort(axis=1)[:, :self.n_neighbors]]
            for j in range(X_c.shape[0]):
                unique, counts = np.unique(dst_m[j], return_counts=True)
                cnt[i * self.C + j, unique] = counts
        return cnt / self.n_neighbors

    def score(self, X, y):
        return (y.astype(int) == self.predict(X)).sum() / y.shape[0]

**IRIS**

In [10]:
iris = datasets.load_iris()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [12]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [13]:
%time clf.fit(X_train, y_train)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 137 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [14]:
%time my_clf.fit(X_train, y_train)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 886 µs


In [15]:
%time clf.predict(X_test)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 4.03 ms


array([1, 1, 1, 2, 2, 0, 0, 0, 1, 0, 2, 2, 1, 0, 2])

In [16]:
%time my_clf.predict(X_test)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 200 ms


array([1, 1, 1, 2, 2, 0, 0, 0, 1, 0, 2, 2, 1, 0, 2])

In [17]:
%time clf.predict_proba(X_test)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.73 ms


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [18]:
%time my_clf.predict_proba(X_test)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.57 ms


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [19]:
clf.score(X_test, y_test)

1.0

In [20]:
my_clf.score(X_test, y_test)

1.0

In [21]:
# Задание 3

from sklearn.model_selection import GridSearchCV


par_grid = {
    "n_neighbors": range(1, 20)
}

pars = dict(cv=5, scoring='accuracy', return_train_score=True)

grid_src = GridSearchCV(clf, par_grid, **pars)
grid_src.fit(X_train, y_train)

my_grid_src = GridSearchCV(my_clf, par_grid, **pars)
my_grid_src.fit(X_train, y_train)

best_n = grid_src.best_estimator_.get_params()['n_neighbors']
my_best_n = my_grid_src.best_estimator_.get_params()['n_neighbors']

scores = grid_src.cv_results_['mean_test_score']
my_scores = my_grid_src.cv_results_['mean_test_score']
plt.plot(par_grid['n_neighbors'], scores, color='blue', label='clf')
plt.plot(par_grid['n_neighbors'], my_scores, color='red', label='my_clf')

print("Sklearn: best n_neighbors = {}".format(best_n))
print("My: best n_neighbors = {}".format(my_best_n))

Sklearn: best n_neighbors = 6
My: best n_neighbors = 6


**MNIST**

In [22]:
mnist = fetch_mldata('MNIST original')

In [23]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.1, stratify=mnist.target)

In [24]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [25]:
%time clf.fit(X_train, y_train)

CPU times: user 1min 37s, sys: 776 ms, total: 1min 38s
Wall time: 1min 39s


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [ ]:
%time my_clf.fit(X_train, y_train)

CPU times: user 448 ms, sys: 680 ms, total: 1.13 s
Wall time: 42.4 s


In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

Задание 5

In [ ]:
# seaborn <— библиотека визуализаций, основанная на matplotlib
# matplotlib <— библиотека визуализаций, содержащая базовые функции и классы для отрисовки графиков
# train_test_split <— функция, случайным образом разбивающая данные на тренировочные и тестовые в заданном соотношении
# Pipelin%load_ext e (from sklearn.pipeline import Pipeline) <— класс, который позволяет создавать наборы действий с 
#     указанными параметрами, совершающимися над входными данными. Унифицирует последовательности обработки данных
# StandardScaler (from sklearn.preprocessing import StandardScaler) <— класс, нормализующий входные данные
# ClassifierMixin <— класс, добавляемый в список наследования классификатора и расширяющий основной класс-предок 
#     стандартными возможностями
# NeighborsBase <— базовый класс, от которого наследуются все классы-классификаторы KNN
# KNeighborsMixin <— класс, расширяющий возможности NeighborsBase и добавляющий две функции: kneighbors(находит k
# ближайших к точке соседей); kneighbors_graph (возвращает матрицу связей между точками(соседи\не соседи) или расстояние
#     между точками)
# SupervisedIntegerMixin <— добавляет метод fit, позволяющий обучить классификатор, используя X как тренировочные
#     данные и y как целевую переменную